# Analisando coleções

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('default')
import numpy as np
import unidecode
import urllib
import string

In [ ]:
stopwords_list = urllib.request.urlopen("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").read().decode()
stopwords_en = set(stopwords_list.split()) 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
s = ['meu primeiro documento é meu', 'meu segundo documento é meu também']
vectorizer = CountVectorizer(ngram_range=(1,4))
x = vectorizer.fit_transform(s)
print(x)
print(vectorizer.vocabulary_)

## Lidando com coleções

In [ ]:
import pandas as pd

df = pd.read_csv('./datasets/IMDB Dataset.csv')
df.head()

In [ ]:
reviews = [s for s in df['review']]
vectorizer = CountVectorizer(stop_words = stopwords_en)
x = vectorizer.fit_transform(reviews)


## Agregando coleções

In [ ]:
# Ideia: somar a frequência de cada palavra ao longo de toda a coleção
x_ = np.sum(x, axis=0)

tuplas = [ (x_[0,vectorizer.vocabulary_[i]], i) for i in vectorizer.vocabulary_.keys() ]
tuplas_ordenadas = sorted(tuplas, reverse=True) # reverse=True pede uma ordenação em ordem decrescente
palavras = [ t[1] for t in tuplas_ordenadas ]
contagens = [ t[0] for t in tuplas_ordenadas ]

print(len([c for c in contagens if c < 50]))


n_palavras = 60
eixo_x = np.arange(n_palavras)
plt.figure(figsize=(14,3))
plt.bar(eixo_x[0:n_palavras], contagens[0:n_palavras])
plt.xticks(eixo_x[0:n_palavras], palavras[0:n_palavras], rotation=90)
plt.show()

## Contar palavras ou detectar a presença de palavras?

In [ ]:
# Ideia: ao inves de somar a frequencia de palavras, verifico:
# (a) a palavra vale 1 se está presente no documento
# (b) a palavra vale 0 caso contrário

reviews = [s for s in df['review']]
vectorizer = CountVectorizer(stop_words = stopwords_en, binary=True)
x = vectorizer.fit_transform(reviews)

x_ = np.mean(x, axis=0)

tuplas = [ (x_[0,vectorizer.vocabulary_[i]], i) for i in vectorizer.vocabulary_.keys() ]
tuplas_ordenadas = sorted(tuplas, reverse=True) # reverse=True pede uma ordenação em ordem decrescente
palavras = [ t[1] for t in tuplas_ordenadas ]
contagens = [ t[0] for t in tuplas_ordenadas ]
eixo_x = np.arange(len(palavras))
n_palavras = 60
plt.figure(figsize=(14,3))
plt.bar(eixo_x[0:n_palavras], contagens[0:n_palavras])
plt.xticks(eixo_x[0:n_palavras], palavras[0:n_palavras], rotation=90)
plt.ylabel('P(word)')
plt.xlabel('Words')
plt.title('Probability of finding a word in a random document from collection')
plt.show()

## Analisando por grupos

In [ ]:
# Passo 1: executar vectorizer ao longo de toda a coleção
reviews = [s for s in df['review']]
vectorizer = CountVectorizer(stop_words = stopwords_en, binary=True)
x = vectorizer.fit_transform(reviews)
x_ = np.mean(x, axis=0)

# Passo 2: escolher somente elementos correspondentes a cada categoria
filtro_pos = df['sentiment']=='positive'
x_pos = x[filtro_pos,:] # Escolho linhas com sentimento positivo
x_pos_ = np.mean(x_pos, axis=0)
filtro_neg = df['sentiment']=='negative'
x_neg = x[filtro_neg,:] # Escolho linhas com sentimento negativo
x_neg_ = np.mean(x_neg, axis=0)

# Passo 3: ordenar palavras de acordo com a soma geral. Vou incluir tambem a contagem nos grupos positivo e negativo!
tuplas = [ (\
            x_[0,vectorizer.vocabulary_[i]],\
            i,\
            x_pos_[0, vectorizer.vocabulary_[i]],\
            x_neg_[0, vectorizer.vocabulary_[i]]\
            ) for i in vectorizer.vocabulary_.keys() ]
tuplas_ordenadas = sorted(tuplas, reverse=True)
palavras = [ t[1] for t in tuplas_ordenadas ]
contagens_pos = [ t[2] for t in tuplas_ordenadas ]
contagens_neg = [ t[3] for t in tuplas_ordenadas ]

eixo_x = np.arange(len(palavras))
n_palavras = 15

plt.figure(figsize=(14,3))
plt.bar(eixo_x[0:n_palavras]-0.125, contagens_pos[0:n_palavras], width=0.25, label='positive')
plt.bar(eixo_x[0:n_palavras]+0.125, contagens_neg[0:n_palavras],  width=0.25, label='negative')
plt.legend()
plt.xticks(eixo_x[0:n_palavras], palavras[0:n_palavras], rotation=90)
plt.ylabel('P(word | class)')
plt.xlabel('Words')
plt.title('Probability of finding a word in a random document from known class')
plt.show()